#  BUILDING A DATABASE FOR CRIME REPORTS
# GOAL
In this guided project, we will build a database for storing data related to crimes that occurred in Boston. This dataset is available in the file **boston.csv.** The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two columns contain the date when the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with latitude and longitude coordinates. The goal of this guided project is to:
1. Create a database named **crimes_db** with a table , **boston_crimes**, with appropriate datatypes for storing the data from the boston.csv file. The table will be created inside a schema named crimes.
2. Create the *readonly* and *readwrite* groups with the appropriate privileges. 
3. Create one user for each of these groups.

In [5]:
import psycopg2
import csv

In [6]:
# Connect to dq database with user dq then disconnect
conn = psycopg2.connect("dbname=postgres user=postgres")
conn.autocommit = True
cur=conn.cursor()
cur.execute("CREATE DATABASE crime_db")
conn.close()

In [7]:
# connect to crime_db database
conn=psycopg2.connect("dbname=crime_db user=postgres")
conn.autocommit = True
cur=conn.cursor()
cur.execute("CREATE SCHEMA crimes")
conn.commit()

In [8]:
# obtaining column names and samples
with open ("boston.csv") as file:
    read_file = csv.reader(file)
    col_headers = next(read_file)
    first_row = next(read_file)
print("Headers:",col_headers)
print("First Row:",first_row)

Headers: ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
First Row: ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


# Creating an Auxiliary Function
Before we create a table to store the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — **get_col_set()** — that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column. This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column
2. Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

In [9]:
# Creating an auxiliary function to identify proper datatypes
def get_col_set(csv_filename, col_index):
    with open('boston.csv') as f:
        next(f)
        reader = csv.reader(f)
        
        distinct_values = set()
        for row in reader:
            distinct_values.add(row[col_index])
            
    return(distinct_values)

for i in range(len(col_headers)):
    distinct_values = get_col_set('boston.csv',i)
    print(col_headers[i], len(distinct_values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


# Finding the Maximum Length
With the function above, we can compute the number of distinct values for each column. Columns with a low number of distinct values tend to be good candidates for enumerated datatypes. We will later:  

1. Choose which datatype to use for each column.
2. Determine the longest word in any column containing textual data.


In [10]:
col_idx = {}
for i in range(len(col_headers)):
    col_idx[col_headers[i]] = i
print(col_idx)

{'incident_number': 0, 'offense_code': 1, 'description': 2, 'date': 3, 'day_of_the_week': 4, 'lat': 5, 'long': 6}


In [11]:
#  compute the maximum length of any value in the description column
col_values = get_col_set('boston.csv', 2)
max_len = 0
for value in col_values:
    max_len = max(max_len, len(value))

print(max_len)

58


In [12]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [13]:
# create enumerated datatype for day of the wek
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# Create the table.
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

In [14]:
# loading the data
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER",file)
cur.execute('SELECT * FROM crimes.boston_crimes;')
data = cur.fetchall()
print(data[0])
print(len(data))

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))
298329


In [15]:
# revoking public privileges
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [16]:
# creating user groups
cur.execute('CREATE GROUP readonly NOLOGIN;')
cur.execute('CREATE GROUP readwrite NOLOGIN;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')
cur.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;')

In [17]:
# creating users
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute('GRANT readwrite TO data_scientist')

In [18]:
conn.close()